In [4]:
import os
from pathlib import Path
import re
import mne
import pyxdf
fname = Path.cwd()
print(fname)

/Users/badralmahouri/Documents/project/npulse/software/notebooks


In [5]:
p = Path.cwd()
repo_root = None
while True:
    if (p / "data").exists() and (p / "notebooks").exists():
        repo_root = p
        break
    if p == p.parent:
        raise FileNotFoundError("Could not find project root containing 'data' and 'notebooks'")
    p = p.parent
data_path = repo_root / "data" / "raw" / "sub-P005" / "emg"

In [6]:
print(data_path)

/Users/badralmahouri/Documents/project/npulse/software/data/raw/sub-P005/emg


In [7]:
def find_bids_emg_files(bids_root, subject, session=None, task=None, run=None):
    """
    Return list of (xdf_path, events_path, json_path) for one subject/session/task.
    """
    subj_dir = os.path.join(bids_root, "data")
    subj_dir = os.path.join(subj_dir, "raw")
    subj_dir = os.path.join(subj_dir, f"sub-{subject}")
    if session:
        subj_dir = os.path.join(subj_dir, f"ses-{session}")
    #if task:
    #    subj_dir = os.path.join(subj_dir, f"task-{task}")
    #if run:
    #    subj_dir = os.path.join(subj_dir, f"run-{run}")
    emg_dir = os.path.join(subj_dir, "emg")
    #print(emg_dir)
    if not os.path.exists(emg_dir):
        raise FileNotFoundError(f"No EMG folder for subject {subject}, session {session}")

    pattern = f"sub-{subject}"
    if session:
        pattern += f"_ses-{session}"
    if task:
        pattern += f"_task-{task}"
    if run:
        pattern += f"_run-{run}"
    pattern += "_eeg_up.xdf"
    
    files = [f for f in os.listdir(emg_dir) if re.match(pattern, f)]
    return [os.path.join(emg_dir, f) for f in files]


In [8]:
find_bids_emg_files(repo_root, subject="P005", session="S002", task="Default", run="001")

FileNotFoundError: No EMG folder for subject P005, session S002

In [53]:
def load_emg_bids(bids_root, subject, session=None, task=None, run=None, label_column="event_label"):
    """
    Load EMG data and labels from a BIDS-compliant directory.
    Returns: X (samples x channels), y (labels)
    """
    xdf_files = find_bids_emg_files(bids_root, subject, session, task, run)
    if len(xdf_files) == 0:
        raise FileNotFoundError(f"No EMG EDF found for task={task}")

    xdf_path = xdf_files[0]  # assuming one run
    base_prefix = xdf_path.replace("_emg.xdf", "")
    events_path = base_prefix + "_events.tsv"

    # --- Load EMG signal ---
    """ raw = mne.io.read_raw_edf(xdf_path, preload=True)
    emg_data = raw.get_data().T  # shape: (n_samples, n_channels)
    sfreq = raw.info["sfreq"] """
    streams, header = pyxdf.load_xdf(xdf_path)
    """ data = streams[0]["time_series"].T
    sfreq = float(streams[0]["info"]["nominal_srate"][0])
    info = mne.create_info(3, sfreq, ["eeg", "eeg", "stim"])
    raw = mne.io.RawArray(data, info)
    raw.plot(scalings=dict(eeg=100e-6), duration=1, start=14) """

    # --- Load events ---
    #events_df = pd.read_csv(events_path, sep="\t")

    #if label_column not in events_df.columns:
    #    raise ValueError(f"Missing {label_column} in events.tsv")

    # Align EMG and events (simple approach: expand labels per time window)
    #y = events_df[label_column].to_numpy()
    y=None
    return streams, header

In [54]:
streams, header = load_emg_bids(repo_root, subject="P005", session="S002", task="Default", run="001")

c:\Users\miski\Desktop\Neuro-X\N-Pulse\npulse-software\data\raw\sub-P005\ses-S002\emg


In [61]:
streams[1]

{'info': defaultdict(list,
             {'name': ['eegoSports 000650'],
              'type': ['EEG'],
              'channel_count': ['71'],
              'nominal_srate': ['1000'],
              'channel_format': ['double64'],
              'source_id': ['eegoSports_000650'],
              'version': ['1.1000000000000001'],
              'created_at': ['7255.1273566999998'],
              'uid': ['055d0277-59f2-42de-8f97-174699be8396'],
              'session_id': ['default'],
              'hostname': ['L-1010036864'],
              'v4address': [None],
              'v4data_port': ['16572'],
              'v4service_port': ['16578'],
              'v6address': [None],
              'v6data_port': ['16572'],
              'v6service_port': ['16578'],
              'desc': [defaultdict(list,
                           {'channels': [defaultdict(list,
                                         {'channel': [defaultdict(list,
                                                       {'label':

In [58]:
streams[0]

{'info': defaultdict(list,
             {'name': ['stimulus_stream'],
              'type': ['Markers'],
              'channel_count': ['1'],
              'channel_format': ['int32'],
              'source_id': ['stimulus_stream_001'],
              'nominal_srate': ['0.000000000000000'],
              'version': ['1.100000000000000'],
              'created_at': ['10014.17207990000'],
              'uid': ['72e7b7f1-2010-4c69-8037-e77479367c40'],
              'session_id': ['default'],
              'hostname': ['L-1027100110'],
              'v4address': [None],
              'v4data_port': ['16572'],
              'v4service_port': ['16572'],
              'v6address': [None],
              'v6data_port': ['16572'],
              'v6service_port': ['16572'],
              'desc': [defaultdict(list,
                           {'experiment': [defaultdict(list,
                                         {'task': ['Upper-limb both arms'],
                                          'vers